In [89]:
### import numpy as np
from tifffile import imread, imsave
import matplotlib.pyplot as plt
from skimage import exposure
from skimage.morphology import white_tophat, disk

# Leer el archivo
img = imread(r"C:\Users\Jon\Documents\Master Biofísica\TFM\PELIS MOSCAS - ORIGINALES\22126 120523 c381ECadGFP UASMbsN300\2-\2- intento.tif")

# Obtener el número total de slides en el eje z y tiempo
num_time, num_z, height, width = img.shape

# Aplicar la operación top-hat a cada slide
for t in range(num_time):
    for z in range(num_z):
        img[t, z] = white_tophat(img[t, z], footprint=disk(2))

img = img.astype(np.float16)
# Definir los rangos de los slides
ranges = [(1, 3), (1, 5), (1, 15), (1, 23)]  # Ajusta los rangos según sea necesario

# Iniciar la proyección final a cero y con tipo float64
final_proj = []

# Loop a través de los tiempos y rangos
for t in range(num_time):
    time_proj = np.zeros_like(img[0][0], dtype=np.float16)
    #projs = []
    for i, (start, end) in enumerate(ranges):
        # Comprobar si el final del rango excede el número de slides
        if end > num_z:
            print(f"El rango {start}-{end} excede el número de slides. Reduciendo el final a {num_z}.")
            end = num_z

        # Realizar la proyección máxima en el rango actual
        max_proj = np.max(img[t][start-1:end], axis=0)  # Restamos 1 al inicio porque Python indexa desde 0

        # Multiplicar la proyección máxima por un factor decreciente
        weighted_proj = max_proj * (1.0 / (i + 1))

        
        # Sumar a la proyección final
        time_proj += weighted_proj

    # Agrega la proyección del tiempo al stack final
    final_proj.append(time_proj)
    
    
# Añadir la proyección ponderada a la lista de proyecciones
        #projs.append(weighted_proj)

    # Calcular la media de las proyecciones ponderadas y añadirla a la proyección final
    #final_proj.append(np.mean(projs, axis=0))

# Convertir la lista en una matriz numpy para guardarla como una imagen
final_proj = np.array(final_proj)

# Ajustar gamma de cada slide en el tiempo
#for t in range(final_proj.shape[0]):
#    final_proj[t] = exposure.adjust_gamma(final_proj[t], gamma=0.5)  # Ajustar el valor de gamma según sea necesario
#final_proj = final_proj * 255

output_file = r"C:\Users\Jon\Documents\Master Biofísica\TFM\PELIS MOSCAS - ORIGINALES\22126 120523 c381ECadGFP UASMbsN300\2-\2-maximos.tif"
imsave(output_file, final_proj)


C:\Users\Jon\AppData\Local\Temp\ipykernel_2036\54714764.py:64: DeprecationWarning: <tifffile.imsave> is deprecated. Use tifffile.imwrite
  imsave(output_file, final_proj)
